In [1]:
from lxml import html

# from bs4 import BeautifulSoup
from bs4 import BeautifulSoup, NavigableString
import time
import datetime
import threading
from apscheduler.schedulers.background import BackgroundScheduler

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.chrome.options import Options

from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec

from selenium.common.exceptions import TimeoutException

import json
import os
import sys

In [2]:
course_schedule = []
NO_WINDOW = False
IMPLICIT_WAIT_TIME = 20
EXPLICIT_WAIT_TIME = 20
WINDOWS_GRID_X = 3
WINDOWS_GRID_Y = 2
SCREEN_WIDTH = 2560
SCREEN_HEIGHT = 1440
# SCREEN_WIDTH = 2048
# SCREEN_HEIGHT = 1152
WINDOW_WIDTH = SCREEN_WIDTH / WINDOWS_GRID_X
WINDOW_HEIGHT = SCREEN_HEIGHT / WINDOWS_GRID_Y

RETRY_INTERVAL = 180

In [3]:
def sync_get_element_by_xpath(driver, xpath):
    wait = WebDriverWait(driver, 15)
    try:
        wait.until(ec.visibility_of_element_located((By.XPATH, xpath)))
    except TimeoutException:
#         print("Get element from xpath:{} Timeout".format(xpath))
        return None
    return driver.find_element_by_xpath(xpath)

def teacher_norm(teacher):
    if not teacher.startswith('-'):
        return teacher
    tmp = teacher.split('\n')[0].split('- ')[1]
    return tmp[:-2]

def search_courses_by_date(date_str, courses):
    results = []
    
    date_str = date_str.strip()
    
    for item in courses:
        if date_str in item['date']:
            results.append(item)
    return results

def search_courses_by_teacher(teacher_str, courses):
    results = []
    
    teacher_str = teacher_str.strip()
    
    for item in courses:
        if teacher_str in item['teacher']:
            results.append(item)
    return results

def search_courses_by_name(name_str, courses):
    results = []
    
    name_str = name_str.strip()
    
    for item in courses:
        if name_str in item['name']:
            results.append(item)
    return results

def search_courses_by_time(time_str, courses):
    results = []
    
    time_str = time_str.strip()
    
    for item in courses:
        if time_str in item['time']:
            results.append(item)
    return results

    
def list_courses_to_be_booked(courses_to_be_booked):
    cur_date = ''
    if len(courses_to_be_booked) <= 0:
        return
    
    for course in courses_to_be_booked:  

        if cur_date != course['date']:
            if cur_date != '':
                print("-----------------------")
            print("Date: {}".format(course['date']))
            print("-----------------------\n")
            cur_date = course['date']
        print("Name: {}".format(course['name']))
        print("Teacher: {}".format(course['teacher']))
        print("Time: {}".format(course['time']))
        print("\n")
        

In [4]:
def create_new_driver():
    #setup option for chrome profile
    # chrome_options = Options()
    # chrome_options.add_argument("user-data-dir=/Users/huohsien/Library/Application Support/Google/Chrome/Default/")
    options = Options()
    
    if NO_WINDOW:
        options = Options()
        options.add_argument('--headless')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-gpu')

    #start web driver
    driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
    driver.implicitly_wait(IMPLICIT_WAIT_TIME)

    wait = WebDriverWait(driver, EXPLICIT_WAIT_TIME)
    return driver, wait

def send_keys(elm, string):
    for c in string:
        elm.send_keys(c)
        
def sign_in(driver, wait):
    
    username = driver.find_element_by_id('ctl00_cphContents_txtUsername')
    password = driver.find_element_by_id('ctl00_cphContents_txtPassword')
    username.send_keys("DN20092361")
    password.send_keys("LP7J")
#     send_keys(username, "DN20092360")
#     send_keys(password, "jj1216")

    wait.until(ec.element_to_be_clickable((By.CLASS_NAME, "submit-button")))
#     wait.until(ec.presence_of_element_located((By.XPATH, "//div[@class='submit-button']/input")))

    sign_in_btn = driver.find_element_by_class_name("submit-button") 
    sign_in_btn.click()      
        

def log_out(driver, wait):
    click_btn = sync_get_element_by_xpath(driver, "//a[contains(text(), 'Logout')]")
#     print("click_btn: ", click_btn)
    if click_btn:
        click_btn.click()
    
def click_book_for_class(driver, wait):
    click_btn = sync_get_element_by_xpath(driver, "//a[contains(text(), 'Book for Class')]")
#     print("click_btn: ", click_btn)
    if click_btn:
        click_btn.click()

def click_book_this_class_now(driver, wait):
    click_btn = sync_get_element_by_xpath(driver, "//a[contains(text(), 'BOOK THIS CLASS NOW')]")
#     print("click_btn: ", click_btn)
    if click_btn:
        click_btn.click()
        return True
    else:
        return False

In [5]:
#launch url
base_url = "https://www.trueclassbooking.com.tw/member/search-class.aspx"

In [6]:
temp = []

# ---------------

with open(r'./class_schedules/10 January - 16 January 2022.json', "r") as read_file:
    course_schedule = json.load(read_file)
# ---------------


c = search_courses_by_date('15', course_schedule)
c = search_courses_by_name('Yin', c)
c = search_courses_by_teacher('Moor', c)
# c = search_courses_by_time('PM', c)
temp.extend(c)

c = search_courses_by_date('15', course_schedule)
c = search_courses_by_name('Flow', c)
c = search_courses_by_teacher('Moor', c)
# c = search_courses_by_time('PM', c)
temp.extend(c)

# c = search_courses_by_date('13', course_schedule)
# c = search_courses_by_name('Go', c)
# c = search_courses_by_teacher('Andy', c)
# c = search_courses_by_time('PM', c)
# temp.extend(c)

courses_to_be_booked = temp

list_courses_to_be_booked(courses_to_be_booked)

Date: 15 Jan Sat
-----------------------

Name: Yin Yoga 陰瑜珈
Teacher: Moorthi
Time: 09:30 - 10:30 AM


Name: Flow Yoga 動瑜珈
Teacher: Moorthi
Time: 11:00 - 12:00 PM




In [7]:
master_counter = 1

In [8]:
def sign_in_once(driver, wait):
#     print("sign_in({},{})".format(driver, wait))
    driver.get(base_url)
    sign_in(driver, wait)

In [9]:
def timestamp_string():

    date_time = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
#     date_time = datetime.datetime.now().strftime("%S")

    return date_time

In [10]:
csv_file_name = "./log_try_booking_a_class_that_was_full-- {}.csv".format(timestamp_string())

In [11]:
class_idx = 1
num_classes = len(courses_to_be_booked)

In [12]:
def reserve_class(driver, wait, course_to_be_booked):
    
    global master_counter
    global class_idx
    global num_classes

    stride =RETRY_INTERVAL / 60 * 15 * num_classes

    book_url = course_to_be_booked['href']
    class_name = course_to_be_booked['name']
    teacher = course_to_be_booked['teacher']
    
#     reserve_class_log_dict = {}
    
#     reserve_class_log_dict['name'] = class_name
    driver.get(book_url)
    
    if click_book_this_class_now(driver, wait):
        
        print("master counter = {}, stride = {}, class_idx = {}".format(master_counter, stride, class_idx))

        log_str = "'{}','{}' by '{}',BOOKED,'{}'\n".format(master_counter, class_name, teacher, datetime.datetime.now())
        print(log_str)
        
        with open(csv_file_name, "a") as write_file:
            write_file.write(log_str)
        num_classes = num_classes - 1
    else:
        
#         print("master counter = {}, stride = {}, class_idx = {}".format(master_counter, stride, class_idx))

        log_str = "'{}','{}' by '{}',FULL,'{}'\n".format(master_counter, class_name, teacher, datetime.datetime.now())
        print(log_str)
        
        if (master_counter - class_idx) % stride == 0:
            
            with open(csv_file_name, "a") as write_file:
                write_file.write(log_str)
        
        next_book_datetime = datetime.datetime.now() + datetime.timedelta(seconds=RETRY_INTERVAL)
        scheduler.add_job(reserve_class, 'date', run_date= next_book_datetime, args=[driver, wait, course_to_be_booked])
        master_counter = master_counter + 1
    
    if class_idx < num_classes:
        class_idx = class_idx + 1
    else:
        class_idx = 1


In [13]:
drivers = []
waits = []
scheduler = BackgroundScheduler()
start = time.time()

#CSV file table header
with open(csv_file_name, "w") as write_file:
            write_file.write("#,Class,Status,Time\n")
        
for idx, course_to_be_booked in enumerate(courses_to_be_booked):

    driver, wait = create_new_driver()
    driver.set_window_position(WINDOW_WIDTH * (idx % 3), WINDOW_HEIGHT * int(idx / 3));
    driver.set_window_size(WINDOW_WIDTH, WINDOW_HEIGHT)

    sign_in_once(driver, wait)
    next_book_datetime = datetime.datetime.now() + datetime.timedelta(seconds=30)
    scheduler.add_job(reserve_class, 'date', run_date= next_book_datetime, args=[driver, wait, course_to_be_booked])
    
# print("Time Elapsed: {:.2f}".format(time.time() - start)) 
scheduler.start()

[WDM] - Current google-chrome version is 97.0.4692
[WDM] - Get LATEST driver version for 97.0.4692
[WDM] - Get LATEST driver version for 97.0.4692
[WDM] - Trying to download new driver from http://chromedriver.storage.googleapis.com/97.0.4692.71/chromedriver_mac64.zip


[WDM] - Driver has been saved in cache [/Users/huohsien/.wdm/drivers/chromedriver/mac64/97.0.4692.71]
[WDM] - Current google-chrome version is 97.0.4692
[WDM] - Get LATEST driver version for 97.0.4692
[WDM] - Driver [/Users/huohsien/.wdm/drivers/chromedriver/mac64/97.0.4692.71/chromedriver] found in cache


 
'1','Yin Yoga 陰瑜珈' by 'Moorthi',FULL,'2022-01-14 07:32:46.788896'

'2','Flow Yoga 動瑜珈' by 'Moorthi',FULL,'2022-01-14 07:32:52.054874'

'3','Yin Yoga 陰瑜珈' by 'Moorthi',FULL,'2022-01-14 07:36:07.456889'

'4','Flow Yoga 動瑜珈' by 'Moorthi',FULL,'2022-01-14 07:36:12.718928'

'5','Yin Yoga 陰瑜珈' by 'Moorthi',FULL,'2022-01-14 07:39:28.145378'

'6','Flow Yoga 動瑜珈' by 'Moorthi',FULL,'2022-01-14 07:39:33.369828'

'7','Yin Yoga 陰瑜珈' by 'Moorthi',FULL,'2022-01-14 07:42:49.311743'

'8','Flow Yoga 動瑜珈' by 'Moorthi',FULL,'2022-01-14 07:42:54.472397'

'9','Yin Yoga 陰瑜珈' by 'Moorthi',FULL,'2022-01-14 07:46:10.767161'

'10','Flow Yoga 動瑜珈' by 'Moorthi',FULL,'2022-01-14 07:46:15.255673'

'11','Yin Yoga 陰瑜珈' by 'Moorthi',FULL,'2022-01-14 07:49:31.955122'

'12','Flow Yoga 動瑜珈' by 'Moorthi',FULL,'2022-01-14 07:49:36.140892'

'13','Yin Yoga 陰瑜珈' by 'Moorthi',FULL,'2022-01-14 07:52:53.179242'

'14','Flow Yoga 動瑜珈' by 'Moorthi',FULL,'2022-01-14 07:52:57.067233'

'15','Yin Yoga 陰瑜珈' by 'Moorthi',FULL,'2022-01-1